In [1]:
import pandas as pd
# import vaex
import numpy as np
import glob
import dask.dataframe as dd
import json
from sklearn.model_selection import train_test_split
import math
import csv
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report, confusion_matrix
import time
import _warnings
import tensorflow as tf
from tqdm import tqdm
import swifter
import argparse
import os

attributes = ['Timestamp', 'canID', 'DLC', 
                           'Data0', 'Data1', 'Data2', 
                           'Data3', 'Data4', 'Data5', 
                           'Data6', 'Data7', 'Flag']

2023-08-08 21:10:11.954884: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-08 21:10:12.006374: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-08 21:10:12.007580: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-08 21:10:12.849151: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Load model and process test data

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

print(sys.path)

['/mnt/hdd2/transformer-entropy-ids/notebooks', '/home/tiendat/miniconda3/envs/piptorchtf/lib/python39.zip', '/home/tiendat/miniconda3/envs/piptorchtf/lib/python3.9', '/home/tiendat/miniconda3/envs/piptorchtf/lib/python3.9/lib-dynload', '', '/home/tiendat/miniconda3/envs/piptorchtf/lib/python3.9/site-packages', '../']


In [3]:
import torch
import torch.nn as nn
import math
from dataPreprocess import DatasetPreprocess
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report, confusion_matrix
import os
import time
import warnings
import torch.nn.functional as F
import time

In [4]:
def draw_confusion(label_y, pre_y, path):
    confusion = confusion_matrix(label_y, pre_y)
    print(confusion)

def write_result(label_y, pre_y, classes_num):
    if classes_num > 2:
        accuracy = accuracy_score(label_y, pre_y)
        macro_precision = precision_score(label_y, pre_y, average='macro')
        macro_recall = recall_score(label_y, pre_y, average='macro')
        macro_f1 = f1_score(label_y, pre_y, average='macro')
        micro_precision = precision_score(label_y, pre_y, average='micro')
        micro_recall = recall_score(label_y, pre_y, average='micro')
        micro_f1 = f1_score(label_y, pre_y, average='micro')
        print('  -- test result: ')
        print('    -- accuracy: ', accuracy)
        print('    -- macro precision: ', macro_precision)
        print('    -- macro recall: ', macro_recall)
        print('    -- macro f1 score: ', macro_f1)
        print('    -- micro precision: ', micro_precision)
        print('    -- micro recall: ', micro_recall)
        print('    -- micro f1 score: ', micro_f1)
        report = classification_report(label_y, pre_y)
        print(report)
    else:
        accuracy = accuracy_score(label_y, pre_y)
        precision = precision_score(label_y, pre_y)
        recall = recall_score(label_y, pre_y)
        f1 = f1_score(label_y, pre_y)
        print('  -- test result: ')
        print('    -- accuracy: ', accuracy)
        print('    -- recall: ', recall)
        print('    -- precision: ', precision)
        print('    -- f1 score: ', f1)
        report = classification_report(label_y, pre_y)
        print(report)

In [5]:
class Config:
    def __init__(self):
        self.model_name = 'Transformer'
        self.slide_window = 1
        self.slsum_count = 8 #int(math.pow(16, self.slide_window))  # 滑动窗口计数的特征的长度 n-gram?
        self.dnn_out_d = 8 # 经过DNN后的滑动窗口计数特征的维度 Dimensions of sliding window count features after DNN 8
        self.head_dnn_out_d = 32 
        self.d_model = self.dnn_out_d + self.head_dnn_out_d # transformer的输入的特征的维度, dnn_out_d + 包头长度 The dimension of the input feature of the transformer, dnn_out_d + header length
        self.pad_size = 29
        self.window_size = 29
        self.max_time_position = 10000
        self.nhead = 5 # ori: 5
        self.num_layers = 5
        self.gran = 1e-6
        self.log_e = 2
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.classes_num = 5 
        self.batch_size = 10
        self.epoch_num = 20
        self.lr = 0.001 #0.00001 learning rate 
        self.train_pro = 0.7  # 训练集比例 Ratio of training set

        self.root_dir = '../road/timesmooth/TFRecord_w29_s29/1/'
        self.model_save_path = '../model/' + self.model_name + '/'
        if not os.path.exists(self.model_save_path):
            os.mkdir(self.model_save_path)
        self.result_file = '/mnt/hdd2/transformer-entropy-ids/result/trans8_performance.txt'

        self.isload_model = False  # 是否加载模型继续训练 Whether to load the model and continue training
        self.start_epoch = 18  # 加载的模型的epoch The epoch of the loaded model
        # self.model_path = '../model/' + self.model_name + '/' + self.model_name + '_model_' + str(self.start_epoch) + '.pth'  # 要使用的模型的路径 path to the model to use
        self.model_path = '../model/Transformer/best_model.pth'
        
class DNN(nn.Module):
    def __init__(self, d_in, d_out):  # config.slsum_count, config.dnn_out_d
        super(DNN, self).__init__()
        self.l1 = nn.Linear(d_in, 128)
        self.l2 = nn.Linear(128, 64)
        self.l3 = nn.Linear(64, d_out)

    def forward(self, x):
        # print('x: ', x.numpy()[0])
        out = F.relu(self.l1(x))
        out = F.relu(self.l2(out))
        out = F.relu(self.l3(out))
        # print('dnn out: ', out.detach().numpy()[0])
        return out

class Time_Positional_Encoding(nn.Module):
    def __init__(self, embed, max_time_position, device):
        super(Time_Positional_Encoding, self).__init__()
        self.device = device

    def forward(self, x, time_position):
        out = x.permute(1, 0, 2)
        out = out + nn.Parameter(time_position, requires_grad=False).to(self.device)
        out = out.permute(1, 0, 2)
        return out

class MyTrans(nn.Module):
    def __init__(self, config):
        super(MyTrans, self).__init__()
        self.dnn = DNN(config.slsum_count, config.dnn_out_d).to(config.device)
        self.head_dnn = DNN(4, config.head_dnn_out_d).to(config.device)
        self.position_embedding = Time_Positional_Encoding(config.d_model, config.max_time_position, config.device).to(config.device)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=config.d_model, nhead=config.nhead).to(config.device)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=config.num_layers).to(config.device)
        self.fc = nn.Linear(config.d_model, config.classes_num).to(config.device)
        self.pad_size = config.pad_size
        self.dnn_out_d = config.dnn_out_d
        self.head_dnn_out_d = config.head_dnn_out_d

    def forward(self, header, sl_sum, mask, time_position):
        dnn_out = torch.empty((sl_sum.shape[0], self.dnn_out_d, 0)).to(config.device)

        for i in range(self.pad_size):
            tmp = self.dnn(sl_sum[:, i, :]).unsqueeze(2)
            dnn_out = torch.concat((dnn_out, tmp), dim=2)
        dnn_out = dnn_out.permute(0, 2, 1)

        head_dnn_out = torch.empty((header.shape[0], self.head_dnn_out_d, 0)).to(config.device)
        for i in range(self.pad_size):
            tmp = self.head_dnn(header[:, i, :]).unsqueeze(2)
            head_dnn_out = torch.concat((head_dnn_out, tmp), dim=2)
        head_dnn_out = head_dnn_out.permute(0, 2, 1)

        x = torch.concat((head_dnn_out, dnn_out), dim=2).permute(1, 0, 2)

        out = self.position_embedding(x, time_position)
        out = self.transformer_encoder(out, src_key_padding_mask=mask)
        out = out.permute(1, 0, 2)
        out = torch.sum(out, 1)
        out = self.fc(out)
        return out


In [6]:
config = Config()
print(config.device)
print(config.model_path)

# seed = 1
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)

train_dataset = DatasetPreprocess(config.root_dir, config.window_size, config.pad_size, config.d_model, config.max_time_position, config.gran, config.log_e, is_train=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=config.batch_size, shuffle=True)

test_dataset = DatasetPreprocess(config.root_dir, config.window_size, config.pad_size, config.d_model, config.max_time_position, config.gran, config.log_e, is_train=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=config.batch_size)
print('finish load data')

# model = torch.load(config.model_path)
# print("Model loaded")

# loss_func = nn.CrossEntropyLoss().to(config.device)
# opt = torch.optim.Adam(model.parameters(), lr=config.lr)

cuda
../model/Transformer/best_model.pth
finish load data


In [7]:
linked_dict = {}

with torch.no_grad():
    for j, train_batch in enumerate(train_loader):
        # print(f"Origin time: {train_batch['oritime']} and length: {len(train_batch['oritime'])}")
        # aa
        canid = []
        message = []
        time_stamp = [] 
        
        for index in range(len(train_batch['oritime'])): 
            time_stamp.extend(train_batch['oritime'][index].tolist())
        
            for msg in train_batch['header'][index]:
                # print(f"{msg}")
                canid.append(''.join(map(str, msg.numpy())))
                # print(f"{''.join(map(str, msg))}")
                
            for msg in train_batch['payload'][index]:
                # print(f"{msg}")
                message.append(msg.numpy())
                # print(f"{''.join(map(str, msg))}")
        # print(canid)
        # print(len(message))
        # print(len(time_stamp))
        
        # Populate the dictionary
        for i, id in enumerate(canid):
            if id in linked_dict:
                linked_dict[id]['payload'].append(message[i])
                linked_dict[id]['timestamp'].append(time_stamp[i])
            else:
                linked_dict[id] = {'payload': [message[i]], 'timestamp': [time_stamp[i]]}
                
for can_id in linked_dict:
    sorted_indices = sorted(range(len(linked_dict[can_id]['timestamp'])), key=lambda k: linked_dict[can_id]['timestamp'][k])
    linked_dict[can_id]['timestamp'] = [linked_dict[can_id]['timestamp'][i] for i in sorted_indices]
    linked_dict[can_id]['payload'] = [linked_dict[can_id]['payload'][i] for i in sorted_indices]


KeyboardInterrupt: 

In [ ]:
count = 0
for can_id in linked_dict:
    count+=1
    time_diffs = [linked_dict[can_id]['timestamp'][i+1] - linked_dict[can_id]['timestamp'][i] for i in range(len(linked_dict[can_id]['timestamp']) - 1)]
    max_time_diff = max(time_diffs)
    min_time_diff = min(time_diffs)
    linked_dict[can_id]['max_time'] = max_time_diff
    linked_dict[can_id]['min_time'] = min_time_diff
    # print(f"CANID: {can_id} with max: {linked_dict[can_id]['max_time']} and min: {linked_dict[can_id]['min_time']}")


In [ ]:
[linked_dict['0230']['payload'][i] for i in range(20)]
print(len(linked_dict['0230']['payload']))
# for i in linked_dict['0230']['payload']:
#     print(i.tolist())
#     aa

17563


In [ ]:
def hamming_distance_int_list(seq1, seq2):
    # Calculate the Hamming distance between two binary sequences represented as lists
    return sum(bin(x ^ y).count('1') for x, y in zip(seq1, seq2))

def all_pair_hamming_distance_int_list(sequence_list):
    
    n = len(sequence_list)
    hamming_distances = {}
    max_distance = 0
    min_distance = 64

    for i in range(n):
        for j in range(i + 1, n):
            distance = hamming_distance_int_list(sequence_list[i].tolist(), sequence_list[j].tolist())
            max_distance = max(max_distance, distance)
            min_distance = min(min_distance, distance)
            hamming_distances[(i, j)] = distance
            # Since Hamming distance is symmetric (H(A, B) == H(B, A)),
            # we can also store the distance for (j, i) pair.
            hamming_distances[(j, i)] = distance
    
    return hamming_distances, max_distance, min_distance

for can_id in linked_dict:
    pair_distances, max_distance, min_distance = all_pair_hamming_distance_int_list(linked_dict[can_id]['payload'])
    # print(f"CANID: {can_id} with max: {linked_dict[can_id]['max_time']} and min: {linked_dict[can_id]['min_time']}")
    linked_dict[can_id]['max_hmc'] = max_distance
    linked_dict[can_id]['min_hmc'] = min_distance
    # for pair, distance in pair_distances.items():
    #     idx1, idx2 = pair
    #     print(f"Hamming distance between {linked_dict[can_id]['payload'][idx1]} and {linked_dict[can_id]['payload'][idx2]}: {distance}")
    print(f"CANID: {can_id} max distance: {max_distance} and min distance: {min_distance}")


NameError: name 'linked_dict' is not defined

In [ ]:
a = ['05141', '02811', '00107', '011210', '00154', '00014', '0585', '00130', '05113', '06914', '0125', '0193', '02014', '0522', '0366', '0580', '0434', '06140', '0498', '0230', '03213', '0153', '0662', '01912', '00130', '02141', '0354', '05141', '0033', '001110', '00014', '02142', '0345', '0430', '0130', '00130', '0033', '00130', '02141', '0354', '05141', '0162', '00107', '0497', '00154', '0125', '05113', '0193', '02014', '0522', '0366', '0580', '0434', '06140', '0498', '0230', '00014', '03121', '06140', '0354', '05141', '02811', '00130', '0033', '061512', '00107', '0239', '00130', '00014', '041513', '05113', '0107', '06914', '0193', '02014', '0522', '02141', '0125', '0354', '0153', '05141', '0662', '01912', '0033', '00130', '0274', '0366', '02811', '0580', '0434', '06140', '00107', '00313', '0498', '0230', '06914', '0125', '00014', '00130', '0033', '05113', '0107', '06140', '02141', '0354', '05141', '03121', '02121', '0162', '00107', '0125', '00014', '0153', '0295', '0662', '01912', '0033', '00014', '00130', '03121', '02141', '0125', '0354', '05141', '0162', '00107', '0107', '0497', '0193', '02014', '0522', '0366', '0580', '0434', '06140', '0498', '0230', '0153', '0295', '0662', '01912', '00130', '0033', '00014', '0274', '0193', '02014', '0522', '0366', '0580', '0434', '02141', '0354', '05141', '06140', '0162', '00107', '0498', '041211', '0280', '0230', '0153', '0662', '01912', '00130', '00014', '06914', '0125', '0033', '0274', '00120', '00130', '06612', '06140', '0295', '0662', '0684', '0636', '02137', '01912', '00130', '0033', '0274', '02141', '0354', '05141', '00120', '00014', '0125', '0162', '00107', '0497', '03144', '06512', '0577', '03121', '06140', '0107', '00130', '0033', '0354', '05141', '02811', '05141', '06914', '0125', '0162', '00107', '06140', '0153', '0662', '01912', '00130', '0107', '0033', '0274', '00120', '00154', '0354', '05141', '02811', '00014', '00107', '0125', '0407', '0193', '02014', '0522', '0366', '0580', '0434', '00130', '02811', '00014', '00107', '011210', '0125', '0193', '02014', '0522', '0366', '0580', '00130', '0033', '0434', '06140', '0498', '0230', '02141', '0354', '05141', '00014', '0162', '00107', '02103', '06914', '0125', '06140', '0153', '0295', '0662', '00120', '00014', '06612', '02614', '00137', '0498', '04129', '01136', '01104', '0230', '0354', '05141', '02811', '00107', '0407', '06140', '00130', '0033', '0125', '00014', '0107', '001110', '0297', '05111', '03213', '02141', '0354', '05141', '0162']


[linked_dict['0230']['timestamp'][i] for i in range(20)]

In [ ]:
label_y = []
pre_y = []
timeCalc = [] 
with torch.no_grad():
    for j, test_sample_batch in enumerate(test_loader):
        start_time = time.time()
        test_header = test_sample_batch['header'].type(torch.FloatTensor).to(config.device)
        test_sl_sum = test_sample_batch['payload'].type(torch.FloatTensor).to(config.device)
        test_mask = test_sample_batch['mask'].to(config.device)
        test_label = test_sample_batch['label'].to(config.device)
        test_time_position = test_sample_batch['time'].to(config.device)
        
        # print(f"TEST HEADER: {test_sample_batch['header']}")
        # print(len(test_sample_batch['label']))
        # aa
        
        # for session in test_sample_batch['header']:
        #     for msg in session:
        #         print(f"Message: {''.join(map(str, msg.numpy()))}")
        #     aa 
        
        test_out = model(test_header, test_sl_sum, test_mask, test_time_position)

        pre = torch.max(test_out, 1)[1].cpu().numpy()
        pre_y = np.concatenate([pre_y, pre], 0)
        label_y = np.concatenate([label_y, test_label.cpu().numpy()], 0)
        end_time = time.time()
        execution_time = end_time - start_time
        timeCalc.append(execution_time)
    write_result(label_y, pre_y, config.classes_num)
    draw_confusion(label_y, pre_y, '')

  -- test result: 
    -- accuracy:  0.9999883322054465
    -- macro precision:  0.9999962205319601
    -- macro recall:  0.9999719881743399
    -- macro f1 score:  0.999984103808873
    -- micro precision:  0.9999883322054465
    -- micro recall:  0.9999883322054465
    -- micro f1 score:  0.9999883322054465
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    105833
         1.0       1.00      1.00      1.00     11236
         2.0       1.00      1.00      1.00     13346
         3.0       1.00      1.00      1.00     19585
         4.0       1.00      1.00      1.00     21412

    accuracy                           1.00    171412
   macro avg       1.00      1.00      1.00    171412
weighted avg       1.00      1.00      1.00    171412

[[105833      0      0      0      0]
 [     1  11235      0      0      0]
 [     0      0  13346      0      0]
 [     1      0      0  19584      0]
 [     0      0      0      0  21412]]


# Entropy algorithm

In [ ]:
def hex_to_int(hex_value):
    return int(hex_value, base=16)

def hex_string_to_array(hex_string):
    return ''.join(map(str, list(map(hex_to_int, hex_string))))

def complete_field(sample):
    if not isinstance(sample['Flag'], str):
        col = 'Data' + str(sample['DLC'])
        sample['Flag'] = sample[col]
        sample[col] = '00'
    return sample

def max_min(lst):
    """
    Finds the maximum and minimum values in a list.
    Returns a tuple containing the maximum and minimum values.
    """
    if len(lst) == 0:
        return None
    
    max_val = lst[0]
    min_val = lst[0]
    for val in lst:
        if val > max_val:
            max_val = val
        elif val < min_val:
            min_val = val
    
    return (max_val, min_val)

def extract_time_interval(time_list):
    """
    Calculates the differences between adjacent elements in an ascending array.
    Returns a list containing all differences.
    """
    if len(time_list) < 2:
        return None
    
    differences = []
    for i in range(len(time_list)-1):
        diff = time_list[i+1] - time_list[i]
        differences.append(diff)
    
    return max_min(differences)

def hamming_distance_int_list(seq1, seq2):
    # Calculate the Hamming distance between two binary sequences represented as lists
    return sum(bin(x ^ y).count('1') for x, y in zip(seq1, seq2))

def all_pair_hamming_distance_int_list(sequence_list):
    n = len(sequence_list)
    hamming_distances = {}
    max_distance = 0
    min_distance = 64

    for i in range(n):
        for j in range(i + 1, n):
            distance = hamming_distance_int_list(sequence_list[i], sequence_list[j])
            max_distance = max(max_distance, distance)
            min_distance = min(min_distance, distance)
            hamming_distances[(i, j)] = distance
            # Since Hamming distance is symmetric (H(A, B) == H(B, A)),
            # we can also store the distance for (j, i) pair.
            hamming_distances[(j, i)] = distance
    
    return hamming_distances, max_distance, min_distance


In [ ]:
def hamming_distance_int(list1, list2):
    if len(list1) != len(list2):
        raise ValueError("Input lists must have the same length.")
    
    distance = sum(a != b for a, b in zip(list1, list2))
    return distance

# Example usage:
list1_example = [216, 0, 0, 137, 0, 0, 0, 0]
list2_example = [216, 85, 0, 139, 0, 0, 0, 0]
distance_value = hamming_distance_int(list1_example, list2_example)
print("Hamming distance between the integer lists:", distance_value)


Hamming distance between the integer lists: 2


In [ ]:
def hamming_distance_int_list(seq1, seq2):
    # Calculate the Hamming distance between two binary sequences represented as lists
    return sum(bin(x ^ y).count('1') for x, y in zip(seq1, seq2))

def all_pair_hamming_distance_int_list(sequence_list):
    n = len(sequence_list)
    hamming_distances = {}
    max_distance = 0
    min_distance = 64

    for i in range(n):
        for j in range(i + 1, n):
            distance = hamming_distance_int_list(sequence_list[i], sequence_list[j])
            max_distance = max(max_distance, distance)
            min_distance = min(min_distance, distance)
            hamming_distances[(i, j)] = distance
            # Since Hamming distance is symmetric (H(A, B) == H(B, A)),
            # we can also store the distance for (j, i) pair.
            hamming_distances[(j, i)] = distance
    
    return hamming_distances, max_distance, min_distance

# Example usage:
binary_sequence_list_example = [
    [216, 0, 0, 138, 0, 0, 0, 0],
    [216, 85, 0, 139, 0, 0, 0, 0],
    [216, 0, 0, 137, 0, 0, 0, 0],
    [216, 85, 0, 140, 0, 0, 0, 0],
    [216, 0, 0, 138, 0, 0, 0, 0]
]

pair_distances, max_distance, min_distance = all_pair_hamming_distance_int_list(binary_sequence_list_example)

# Print the Hamming distances for all pairs
for pair, distance in pair_distances.items():
    idx1, idx2 = pair
    print(f"Hamming distance between {binary_sequence_list_example[idx1]} and {binary_sequence_list_example[idx2]}: {distance}")
    
print(f"Max distance: {max_distance} and min distance: {min_distance}")


Hamming distance between [216, 0, 0, 138, 0, 0, 0, 0] and [216, 85, 0, 139, 0, 0, 0, 0]: 5
Hamming distance between [216, 85, 0, 139, 0, 0, 0, 0] and [216, 0, 0, 138, 0, 0, 0, 0]: 5
Hamming distance between [216, 0, 0, 138, 0, 0, 0, 0] and [216, 0, 0, 137, 0, 0, 0, 0]: 2
Hamming distance between [216, 0, 0, 137, 0, 0, 0, 0] and [216, 0, 0, 138, 0, 0, 0, 0]: 2
Hamming distance between [216, 0, 0, 138, 0, 0, 0, 0] and [216, 85, 0, 140, 0, 0, 0, 0]: 6
Hamming distance between [216, 85, 0, 140, 0, 0, 0, 0] and [216, 0, 0, 138, 0, 0, 0, 0]: 6
Hamming distance between [216, 0, 0, 138, 0, 0, 0, 0] and [216, 0, 0, 138, 0, 0, 0, 0]: 0
Hamming distance between [216, 0, 0, 138, 0, 0, 0, 0] and [216, 0, 0, 138, 0, 0, 0, 0]: 0
Hamming distance between [216, 85, 0, 139, 0, 0, 0, 0] and [216, 0, 0, 137, 0, 0, 0, 0]: 5
Hamming distance between [216, 0, 0, 137, 0, 0, 0, 0] and [216, 85, 0, 139, 0, 0, 0, 0]: 5
Hamming distance between [216, 85, 0, 139, 0, 0, 0, 0] and [216, 85, 0, 140, 0, 0, 0, 0]: 3
Ha

In [ ]:
import numpy as np

def conditional_entropy(y, x):
    # Convert input to numpy arrays if they are not already
    y = np.array(y)
    x = np.array(x)
    
    # Get unique values and their counts for X and Y
    unique_x, counts_x = np.unique(x, return_counts=True)
    unique_y, counts_y = np.unique(y, return_counts=True)
    
    # Initialize conditional entropy
    h_y_given_x = 0.0
    
    # Calculate conditional entropy
    for xi, cx in zip(unique_x, counts_x):
        # Get the subset of Y corresponding to xi in X
        y_given_xi = y[x == xi]
        
        # Get unique values and their counts for the subset of Y given xi in X
        unique_y_given_xi, counts_y_given_xi = np.unique(y_given_xi, return_counts=True)
        
        # Calculate conditional probabilities P(Y|X) for xi
        p_y_given_xi = counts_y_given_xi / cx
        
        # Calculate conditional entropy contribution for xi
        h_y_given_xi = np.sum(-p_y_given_xi * np.log2(p_y_given_xi))
        
        # Weighted sum based on P(X=xi)
        h_y_given_x += h_y_given_xi * (cx / len(x))
    
    return h_y_given_x

# Example usage:
# Assuming X and Y are two discrete random variables in the form of lists or arrays
X = [1, 1, 1, 0, 1, 1, 0, 0]
Y = [1, 0, 0, 1, 0, 1, 0, 0]

# Calculate H(Y|X)
conditional_entropy_value = conditional_entropy(Y, X)
print("Conditional Entropy H(Y|X) =", conditional_entropy_value)


Conditional Entropy H(Y|X) = 0.9512050593046015


In [ ]:
0.01722097396850586-0.017220020294189453

9.5367431640625e-07